<h2>Agentic RAG App: Groq LLM</h2>

<h3>LLM

In [1]:
# LangChain has a Groq chat model integration:
from langchain_groq import ChatGroq

/Users/cameroncochrane/Data Projects/Agentic-RAG/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the API groq API key:
from pathlib import Path
# Bespoke API key loader functions:
from load_keys import *
import os

In [3]:
p = Path("keys/groq.json")
groq_api_key = load_groq_key(p)
os.environ['GROQ_API_KEY'] = groq_api_key

In [4]:
# Initialise the model:
model = ChatGroq(
            model="openai/gpt-oss-120b",
            temperature=0.2,
            max_retries=2,
        )

In [5]:
initial_message = [("human","hello, how are you?")]
answer_1 = model.invoke(initial_message) # .invoke to send a message to the model.

In [6]:
extracted_message = answer_1.content # x.content returns the message for the user.
print(extracted_message)

Hello! I'm doing great, thank you for asking. How can I assist you today?


In [7]:
internet_message = [("human","Do you have internet access?")]
internet_answer = model.invoke(internet_message)
print(internet_answer.content)

I don’t have the ability to browse the web or pull in real‑time information. My responses are based solely on the knowledge I was trained on, which goes up to September 2021 (with a limited update through 2024). If you need the most current data, I recommend checking a reliable source directly.


In [8]:
def load_llm(model_type:str = 'openai/gpt-oss-120b',api_key_path:str = "keys/groq.json"):
    
    p = Path(api_key_path)

    groq_api_key = load_groq_key(p)
    os.environ['GROQ_API_KEY'] = groq_api_key
    
    model = ChatGroq(
            model=model_type,
            temperature=0.2,
            max_retries=2,
        )
    
    return model

In [9]:
def give_message(model,message):

    full_answer = model.invoke(message)
    extracted_answer = full_answer.content

    return extracted_answer

In [10]:
model_1 = load_llm()
extracted_answer = give_message(model_1,"What are some of your use cases?")
print(extracted_answer)

Sure! I’m a versatile AI language model, and people use me for a wide variety of tasks across many domains. Here are some of the most common (and a few creative) use cases:

---

## 1. **Writing & Editing**
| Task | How I help |
|------|------------|
| **Drafting emails, letters, or memos** | Generate professional or casual drafts, suggest subject lines, and polish tone. |
| **Blog posts & articles** | Outline structures, write full drafts, add headings, SEO keywords, and citations. |
| **Creative writing** | Produce short stories, poems, scripts, character bios, or plot twists. |
| **Proofreading & grammar checks** | Spot typos, improve clarity, suggest style changes, and enforce house style guides. |
| **Resume & cover letter** | Tailor content to specific job postings, highlight achievements, and format for ATS. |

---

## 2. **Learning & Tutoring**
| Subject | What I can do |
|---------|---------------|
| **Academic subjects** (math, physics, chemistry, history, etc.) | Explain con

In [11]:
# We now have the basic llm functionality sorted. Can move onto adding agents with CrewAI

<h4>CrewAI

In [12]:
from crewai import Agent, Task, Crew, Process, LLM
from load_keys import *

In [13]:
openai_key_path = Path('keys/open_ai.json')
openai_key = load_openai_key(openai_key_path)
os.environ['OPENAI_API_KEY'] = openai_key

In [20]:
def build_crew(llm):
    researcher = Agent(
        role="Researcher",
        backstory="",
        goal="Retrieve and compile evidence using the provided LLM as the sole knowledge source (do NOT call external/local tools).",
        llm=llm,
        tools=[],
        verbose=False,
    )

    writer = Agent(
        role="Content Writer",
        backstory="",
        goal="Write a structured answer grounded in the evidence produced by the Researcher (LLM).",
        llm=llm,
        tools=[],
        verbose=False,
    )

    critic = Agent(
        role="Reviewer",
        backstory="",
        goal="Check the answer for unsupported claims, missing info, and clarity against the Researcher's evidence.",
        llm=llm,
        tools=[],
        verbose=False,
    )

    t1 = Task(
        description=(
            "Research the user's query using only the LLM as the knowledge base. Return a bullet list of findings."
        ),
        expected_output="A bullet list of findings derived from the LLM only.",
        agent=researcher,
    )

    t2 = Task(
        description=(
            "Write the final answer in Markdown. Use the Researcher's findings as the source."
        ),
        expected_output="A Markdown response mapped to the Researcher's findings.",
        agent=writer,
        context=[t1],
    )

    t3 = Task(
        description=(
            "Critique the answer against the Researcher's response. List issues and provide fix instructions."
        ),
        expected_output="A list of issues found plus concrete fix instructions.",
        agent=critic,
        context=[t1, t2],
    )

    return Crew(
        agents=[researcher, writer, critic],
        tasks=[t1, t2, t3],
        process=Process.sequential,
        verbose=False,
    )

llm_for_crew = LLM(
    model="groq/openai/gpt-oss-120b",
    api_key=os.environ["GROQ_API_KEY"],
    temperature=0.2,
)

crew = build_crew(llm=llm_for_crew)


In [ ]:
# Using the crew:
user_query = "How old is the planet?"

result = crew.kickoff(inputs={"query": user_query})
print(result)
# The critic and writer agents are working. It doesn't appear to be the same for the researcher agent.

**Issues Identified**

1. **Missing Request for Constraints**  
   - The Researcher’s prompt explicitly asks for “any particular focus areas, sub‑questions, or constraints.” The answer does not request this information, leaving a potential gap in the user’s specifications.

2. **No Mention of Output Format**  
   - The Researcher states that the findings will be delivered as a “bullet‑point list.” The answer fails to confirm that this is the intended format, which could lead to a mismatch in expectations.

3. **Lack of Confirmation About Source Limitation**  
   - The Researcher specifies that the findings will be compiled “using the language model as the sole source.” The answer does not acknowledge or repeat this limitation, which is a key part of the request.

4. **Redundant or Overly Formal Language**  
   - The answer repeats the same request verbatim without adding any clarification or friendly framing. This can feel robotic and may reduce user engagement.

5. **No Assurance of F

╭────────────────────────── Trace Batch Finalization ──────────────────────────╮
│ ✅ Trace batch finalized with session ID:                                    │
│ 8dd14c6e-7634-4a6c-a92e-9c9efdc01171                                         │
│                                                                              │
│ 🔗 View here:                                                                │
│ https://app.crewai.com/crewai_plus/ephemeral_trace_batches/8dd14c6e-7634-4a6 │
│ c-a92e-9c9efdc01171?access_code=TRACE-08ccacbb8c                             │
│ 🔑 Access Code: TRACE-08ccacbb8c                                             │
╰──────────────────────────────────────────────────────────────────────────────╯
